In [1]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
        
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
    
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]
    
sys.meta_path.append(NotebookFinder())

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable   

sys.path.append("../Preprocessor")
import format_module

import rnn
import naivebayesian
import cnn

importing Jupyter notebook from rnn.ipynb


In [3]:
import pickle

def save_object(obj, filename):
    with open(filename, 'wb') as output:
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def load_object(filename):
    with open(filename, 'rb') as input:
        return pickle.load(input)

In [4]:
input_size = 100  # word2vec k size
batch_size = 10

In [9]:
class classifierModule(nn.Module):
    def __init__(self):
        super(classifierModule, self).__init__()
        self.rnn_model = rnn.RNN_model(input_size)
        self.nb_model = naivebayesian.NaiveBayesianDB()
        self.cnn_model = cnn.ConvNet(input_size)
        #TODO
        
    def encoder(self, formattedList):
        length = len(formattedList)
        contextList = [formattedList[i].context for i in range(length)]

        lengths = torch.LongTensor([len(contextList[i]) for i in range(length)])
        max_len = torch.max(lengths)
        
        data = np.zeros((length, max_len, input_size))

        for i in range(length):
            context = contextList[i]
            if not (context.size == 0):
                data[i, :context.shape[0],:] = context
            else:
                lengths[i] = 1
            i+=1
            
        return self.sort_batch(torch.FloatTensor(data), formattedList, lengths)
        
    def sort_batch(self, context, formatted, seq_len):
        batch_size = context.size(0)
        sorted_seq_len, sorted_idx = seq_len.sort(0, descending = True)
        
        sorted_context = context[sorted_idx]
        sorted_formatted = [formatted[i] for i in sorted_idx]

        for f in sorted_formatted:
            print(len(f.context))
        
        return Variable(sorted_context), sorted_formatted, sorted_seq_len
    
    def resize_input(self, input):
        list_ = list()
        for i in range(0, len(input), batch_size):
            list_.append(input[i:i+batch_size])
        return list_
        
    def forward(self, formatted_list, hidden=None):
        context, formatted, lengths = self.encoder(bl)
        
        print("rnn_out : ", self.rnn_model(context, lengths), "\n\n\n")
        print("cnn_out : ", self.cnn_model(context), "\n\n\n")
        #self.nb_model.naive_bayes_FRlist(formatted)

In [10]:
formatted_list = load_object("../Preprocessor/save_formatted_review.pkl")
test_classifier = classifierModule()

test_classifier.nb_model.add_FRlist(formatted_list)

batch_list = test_classifier.resize_input(formatted_list)
for bl in batch_list:
    test_classifier(bl)

for param in test_classifier.parameters():
     print(type(param.data), param.size())

137
120
52
42
34
32
27
25
20
9
rnn_out :  Variable containing:
-0.1098  0.0234 -0.2328  0.0208  0.1361  0.1275 -0.1748 -0.0896 -0.0864 -0.1633
-0.1187 -0.0083 -0.2349  0.0163  0.1354  0.1380 -0.1538 -0.0644 -0.0751 -0.1645
-0.0928  0.0011 -0.2306  0.0183  0.1756  0.1422 -0.1834 -0.1066 -0.0701 -0.1644
-0.0980  0.0118 -0.2446  0.0273  0.1193  0.1290 -0.2118 -0.1409 -0.0648 -0.1693
-0.1098  0.0179 -0.2352  0.0235  0.1448  0.1385 -0.2135 -0.1431 -0.0693 -0.1624
-0.0893  0.0169 -0.2374  0.0204  0.1653  0.1319 -0.2140 -0.1340 -0.0709 -0.1616
-0.1003  0.0246 -0.2366  0.0199  0.1645  0.1356 -0.1653 -0.0946 -0.1024 -0.1680
-0.0908  0.0255 -0.2399  0.0268  0.1619  0.1309 -0.2371 -0.1675 -0.0649 -0.1633
-0.0920  0.0352 -0.2316  0.0337  0.1489  0.1198 -0.2108 -0.1650 -0.0835 -0.1694
-0.0954  0.0162 -0.2338  0.0115  0.1505  0.1337 -0.1733 -0.0930 -0.0833 -0.1655
[torch.FloatTensor of size 10x10]
 



torch.Size([1370, 1])
torch.Size([10, 137])
[[ 0.          1.          0.         ...,  0.        

In [37]:
a = ["ㄱ", "ㄴ", "ㄷ", "ㄹ", "ㅁ"]
b = [4, 3, 2, 1, 0]
c = [4, 2, 1, 0, 3]

d = [x for _,x in sorted(zip(c,a))]
print(d)
list.reverse(d)
print(d)

['ㄹ', 'ㄷ', 'ㄴ', 'ㅁ', 'ㄱ']
['ㄱ', 'ㅁ', 'ㄴ', 'ㄷ', 'ㄹ']


In [10]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a_ = list()
for i in range(0, len(a), 3):
    a_.append(a[i:i+3])
print(a_)

[[1, 2, 3], [4, 5, 6], [7, 8, 9], [10]]
